In [19]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def circuit(weight):
    #qml.FermionicSingleExcitation(weight, wires=[0,3])
    qml.DoubleExcitation(weight, wires=[0,1,2,3])
    qml.SingleExcitation(weight, wires=[1,2])
    
    return qml.state()

weight = 5.0
print(circuit(weight))
print(qml.draw(circuit, level="device", max_length=1000)(weight))


[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
0: ─╭G²(5.00)──────────┤ ╭State
1: ─├G²(5.00)─╭G(5.00)─┤ ├State
2: ─├G²(5.00)─╰G(5.00)─┤ ├State
3: ─╰G²(5.00)──────────┤ ╰State


In [16]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

dev = qml.device('default.qubit', wires=8)
#hf_state = [1,1,1,1,0,0,0,0]
hf_state = [0,0,0,0,1,1,1,1]

@qml.qnode(dev)
def circuit(weight):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    #qml.FermionicSingleExcitation(weight, wires=[0,2])
    qml.SingleExcitation(weight, wires=[0,4])
    #qml.FermionicDoubleExcitation(weight, wires1=[0,1], wires2=[2,3])
    #qml.DoubleExcitation(weight, wires=[0,1,2,3])
    #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    return qml.state()

weight = 0.9
print(circuit(weight))
print(qml.draw(circuit, level="device", max_length=1000)(weight))


[0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.9004471 +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.      

In [ ]:
#-0.43496553  0.9004471 - 4th line

In [2]:
import numpy as np
import pennylane as qml
from scipy.optimize import minimize

import pennylane as qml

from pennylane import qchem
from pennylane import numpy as np




X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

symbols = ["H","H","H","H"]
r_bohr = 3.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method = "openfermion")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
qubits = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")



dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))
    

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

# Define the Ash excitation structure (as per your example)
#ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4], [2,6]]
#ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [3, 7], [0,4]]
#ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1, 5]]

ash_excitation = [[2,6],[1, 2, 4, 7],[0,1,4,5],[0, 3, 5, 6],[2, 3, 6, 7]]
params = np.zeros(len(ash_excitation), requires_grad=True)

dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
@qml.qnode(dev)
def circuit(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            qml.FermionicDoubleExcitation(weight = params[i], wires1=excitations[2:][::-1], wires2=excitations[:2][::-1])
        else:
            print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            qml.FermionicSingleExcitation(weight= params[i], wires = ash_excitation[i])
    return qml.expval(H)


#dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
#@qml.qnode(dev)
def cost_fn(params, ash_excitation, hf_state, qubits, H):
    # Compute the energy
    energy = circuit(params, ash_excitation, hf_state, qubits, H)
    
    # Compute the gradient of the energy with respect to the parameters
    #grad = qml.grad(circuit)(params, ash_excitation, hf_state, qubits, H)
    
        # Calculate gradient norm (L2 norm)
    #grad_norm = np.linalg.norm(grad)
    
    # Calculate maximum gradient
    #grad_max = np.max(np.abs(grad))
    
    # Debugging: Print the gradient information
    print(f"Energy: {energy}")
    #print(f"Gradient: {grad}")
    #print(f"Gradient Norm: {grad_norm}")
    #print(f"Maximum Gradient: {grad_max}")
    return energy



params_initial = np.zeros(len(ash_excitation))
#bounds = ([(0, 0.97)] * len(ash_excitation))

# Function to minimize
def minimize_function(params):
    energy= cost_fn(params, ash_excitation, hf_state, qubits, H)
    return energy

# Using scipy.optimize.minimize with L-BFGS-B
result = minimize(
    fun=minimize_function,   # The cost function
    x0=params_initial,       # Initial guess
    jac=True,                 # Enable gradient computation
    method='L-BFGS-B',        # Use L-BFGS-B optimizer
    #bounds=bounds,       # Use L-BFGS-B optimizer
    options={'disp': True, 'maxiter': 100, 'gtol' : 1e-12}  # Display options and max iterations
)

# Extract the result
optimized_params = result.x
minimized_energy = result.fun
Gradient = result.jac   

print(f"Optimized parameters: {optimized_params}")
print(f"Minimized energy: {minimized_energy}")


The original hamiltonian is -1.107840826225262 * I(0) + 0.050636412339764195 * Z(0) + 0.0020229573310193345 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.0020229573310193345 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.05063641233976422 * Z(1) + 0.0020229573310193358 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5)) + 0.0020229573310193358 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5)) + 0.04498894793099835 * Z(2) + 0.0021181739685786644 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.0021181739685786644 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.04498894793099837 * Z(3) + 0.002118173968578662 * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + 0.002118173968578662 * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + 0.038410202743306296 * Z(4) + 0.03841020274330629 * Z(5) + 0.03231262682525608 * Z(6) + 0.03231262682525608 * Z(7) + 0.07316430147430236 * (Z(0) @ Z(1)) + 0.0088999118249377 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.0088999118249377 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.04512667701466127 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.04512667701466127 * (Y(0) @ Y

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

## Adaptive optimizer

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

#symbols = ["H", "H", "H", "H"]
#geometry = np.array([[0.0, 0.0, 5.66917797],
#                     [0.0, 0.0, 11.33835593],
#                     [0.0, 0.0, 17.0075339],
#                     [0.0, 0.0, 22.67671186]], requires_grad=False)
#H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge = 0)

symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")

n_electrons = 4
singles, doubles = qml.qchem.excitations(n_electrons, qubits)
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
operator_pool = doubles_excitations + singles_excitations
print(len(operator_pool))

hf_state = qml.qchem.hf_state(n_electrons, qubits)
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit():
    qml.BasisState(hf_state, wires=range(qubits))
    return qml.expval(H)

print(circuit())

opt = qml.AdaptiveOptimizer(param_steps = 10)
for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool, drain_pool=False)
    print('Energy:', energy)
    print(qml.draw(circuit, show_matrices=False)())
    print('Largest Gradient:', gradient)
    print()
    if gradient < 1e-4:
        break

26
-1.216023859130328


/Users/s.poyyapakkam/miniconda3/envs/sri/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:209: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


Energy: -1.216023859130328
0: ─╭|Ψ⟩───────────┤ ╭<𝓗>
1: ─├|Ψ⟩───────────┤ ├<𝓗>
2: ─├|Ψ⟩─╭G²(0.74)─┤ ├<𝓗>
3: ─├|Ψ⟩─├G²(0.74)─┤ ├<𝓗>
4: ─├|Ψ⟩─│─────────┤ ├<𝓗>
5: ─├|Ψ⟩─│─────────┤ ├<𝓗>
6: ─├|Ψ⟩─├G²(0.74)─┤ ├<𝓗>
7: ─╰|Ψ⟩─╰G²(0.74)─┤ ╰<𝓗>
Largest Gradient: 0.17173037308522168

Energy: -1.3246235594262918
0: ─╭|Ψ⟩───────────╭G²(0.85)─┤ ╭<𝓗>
1: ─├|Ψ⟩───────────│─────────┤ ├<𝓗>
2: ─├|Ψ⟩─╭G²(0.74)─│─────────┤ ├<𝓗>
3: ─├|Ψ⟩─├G²(0.74)─├G²(0.85)─┤ ├<𝓗>
4: ─├|Ψ⟩─│─────────│─────────┤ ├<𝓗>
5: ─├|Ψ⟩─│─────────├G²(0.85)─┤ ├<𝓗>
6: ─├|Ψ⟩─├G²(0.74)─╰G²(0.85)─┤ ├<𝓗>
7: ─╰|Ψ⟩─╰G²(0.74)───────────┤ ╰<𝓗>
Largest Gradient: 0.2053857441406627

Energy: -1.4674482719721427
0: ─╭|Ψ⟩───────────╭G²(0.85)─╭G²(0.91)─┤ ╭<𝓗>
1: ─├|Ψ⟩───────────│─────────├G²(0.91)─┤ ├<𝓗>
2: ─├|Ψ⟩─╭G²(0.74)─│─────────│─────────┤ ├<𝓗>
3: ─├|Ψ⟩─├G²(0.74)─├G²(0.85)─│─────────┤ ├<𝓗>
4: ─├|Ψ⟩─│─────────│─────────├G²(0.91)─┤ ├<𝓗>
5: ─├|Ψ⟩─│─────────├G²(0.85)─╰G²(0.91)─┤ ├<𝓗>
6: ─├|Ψ⟩─├G²(0.74)─╰G²(0.85)───────────┤ ├<𝓗>
7: ─╰|Ψ⟩─╰G²(0.74)────